# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()
city_data_df.set_index(['City_ID'], inplace=True)
city_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,grytviken,-54.2811,-36.5092,0.90,74,52,3.04,GS,1704157015
1,edinburgh of the seven seas,-37.0676,-12.3116,16.57,90,100,2.13,SH,1704157016
2,atafu village,-8.5421,-172.5159,28.76,68,100,1.31,TK,1704157017
3,puerto colombia,10.9878,-74.9547,27.05,79,11,11.90,CO,1704157018
4,la sarre,48.8002,-79.1996,-5.18,73,98,5.59,CA,1704156899
...,...,...,...,...,...,...,...,...,...
590,okhotsk,59.3833,143.3000,-26.04,79,100,2.53,RU,1704157604
591,pangody,65.8500,74.4859,-30.55,93,9,4.25,RU,1704157605
592,filadelfia,39.9523,-75.1638,6.23,76,100,0.45,US,1704157315


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_cities_plot = city_data_df.hvplot.points(
'Lng',
'Lat',
geo = True,
tiles = "EsriStreet",
frame_width = 800,
frame_height = 600,
size = 'Humidity',
scale = 0.6,
color = 'City')

# Display the map
map_cities_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp_clean = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21)]
wind_speed_clean = max_temp_clean[max_temp_clean['Wind Speed'] < 4.5]
zero_cloudiness = wind_speed_clean[wind_speed_clean['Cloudiness'] < 10]
ideal_cities = zero_cloudiness

# Drop any rows with null values
ideal_cities.dropna()

# Display sample data
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
12,rabigh,22.7986,39.0349,23.18,60,0,2.96,SA,1704157025
25,kerikeri,-35.2268,173.9474,23.28,67,4,3.52,NZ,1704157035
48,le vauclin,14.5452,-60.8388,25.03,83,0,0.51,MQ,1704157058
96,ippy,6.2679,21.2247,23.35,28,4,2.12,CF,1704157105
123,al ghayzah,16.2079,52.1760,22.23,83,2,1.61,YE,1704157130
124,tauranga,-37.6861,176.1667,23.97,59,8,3.18,NZ,1704157130
158,sohar,24.3643,56.7468,22.34,60,0,3.27,OM,1704157161
201,marquelia,16.5833,-98.8167,25.37,67,6,2.94,MX,1704157199
286,arica,-18.4750,-70.3042,22.75,73,0,3.09,CL,1704157308


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
12,rabigh,SA,22.7986,39.0349,60,
25,kerikeri,NZ,-35.2268,173.9474,67,
48,le vauclin,MQ,14.5452,-60.8388,83,
96,ippy,CF,6.2679,21.2247,28,
123,al ghayzah,YE,16.2079,52.1760,83,
124,tauranga,NZ,-37.6861,176.1667,59,
158,sohar,OM,24.3643,56.7468,60,
201,marquelia,MX,16.5833,-98.8167,67,
286,arica,CL,-18.4750,-70.3042,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

 
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params) 

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rabigh - nearest hotel: S.A. TALKE Villa
kerikeri - nearest hotel: Avalon Resort
le vauclin - nearest hotel: Hôtel Macabou
ippy - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
tauranga - nearest hotel: No hotel found
sohar - nearest hotel: No hotel here.
marquelia - nearest hotel: No hotel found
arica - nearest hotel: Hotel Andalucía 
san luis de la loma - nearest hotel: No hotel found
vryburg - nearest hotel: International Hotel
gisborne - nearest hotel: Portside Hotel
estancia - nearest hotel: Jardim
hohoe - nearest hotel: Grand Hotel
upington - nearest hotel: Die Eiland Holiday Ressort
concepcion del uruguay - nearest hotel: Hostería Mitre949
salalah - nearest hotel: Muscat International Hotel
sur - nearest hotel: Sur Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
12,rabigh,SA,22.7986,39.0349,60,S.A. TALKE Villa
25,kerikeri,NZ,-35.2268,173.9474,67,Avalon Resort
48,le vauclin,MQ,14.5452,-60.8388,83,Hôtel Macabou
96,ippy,CF,6.2679,21.2247,28,No hotel found
123,al ghayzah,YE,16.2079,52.1760,83,فندق تاج العرب
124,tauranga,NZ,-37.6861,176.1667,59,No hotel found
158,sohar,OM,24.3643,56.7468,60,No hotel here.
201,marquelia,MX,16.5833,-98.8167,67,No hotel found
286,arica,CL,-18.4750,-70.3042,73,Hotel Andalucía


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "EsriStreet",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    scale = 0.6,
    color = 'City',
    hover_cols=['Country', 'Hotel Name']
)

# Display the map
hotel_map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)